# Step 1 Dev Set Labels

In [ ]:
def step1_dev_set_labels(*, step1_iter, k):
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from peft import prepare_model_for_kbit_training
    from PersonalizedNewsCat import dev_loader
    from ProgressPrinter import ProgressPrinter
    from transformers import T5ForConditionalGeneration
    import torch
    import warnings
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(8675309)

    dev = dev_loader(batch_size=2)

    t5 = prepare_model_for_kbit_training(T5ForConditionalGeneration.from_pretrained('google/flan-t5-xxl', load_in_8bit=True))
    taskllm_model_id = f'User_keq{k}_t5xxl_step1_iter{step1_iter}'
    t5.load_adapter(taskllm_model_id, 'raw_taskllm')
    t5.load_adapter(taskllm_model_id, 'ema_taskllm')
    
    taskllm = TaskLLM(t5=t5, adapter_suffix="taskllm", model_id=taskllm_model_id, choices=dev.choices)

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]

    def make_prior(profile):
        from math import log

        c = [1]*len(dev.choices)
        for v in profile:
            c[dev.choices.index(v['category'])] += 1
        n = sum(c)

        return [ log(cnt) - log(n) for cnt in c ]
    
    print(f'*** step1_iter: {step1_iter} ***')

    devgolds = []
    with ProgressPrinter(f'{k} acc (dev)') as printer, warnings.catch_warnings():
        warnings.filterwarnings("ignore", ".*MatMul8bitLt:.*")
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        for examples, labels in dev:
            with torch.no_grad():
                prior = torch.Tensor([ make_prior(ex['profile']) for ex in examples ]).to(device)
                texts_to_embed = [ [ text[:256]
                                     for text in (' '.join(ex['article'].split()), )
                                   ] +
                                   [ text[:256]
                                     for v in ex['profile']
                                     for text in (' '.join(v['text'].split()), )
                                   ]
                                   for ex in examples
                                 ]
                embeddings = torch.cat(inner_batch(func = dev.embed,
                                                   inner_batch_size = 128,
                                                   inputs = (sum(texts_to_embed, []),)
                                                  ),
                                       dim=0)
                splits = cumsum(map(len, texts_to_embed))
                indices = [ torch.topk(embeddings[a,:] @ embeddings[a+1:b,:].T, dim=0, k=safek).indices
                            for a, b in zip(splits, splits[1:])
                            for safek in (max(0, min(k, b-a-1)),)
                          ]
                prompts = [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in index.to('cpu').tolist() ])
                            for ex, index in zip(examples, indices) ]
                targets = [ dev.choices.index(label) for label in labels ]
                guesses = taskllm.predict(prompts, prior=prior).argmax(dim=1)
                targets = torch.Tensor(targets).long().to(guesses.device)
                emaacc = (guesses == targets).float().mean().item()

                for ex, guess in zip(examples, guesses):
                    devgolds.append({ 'id': ex['id'], 'output': dev.choices[guess] })

            printer.addobs(emaacc)

    with open(f'lamp2u_xxl_step1_dev_golds.json', 'w') as jsonfile:
        json.dump({ 'task': 'LaMP_2', 'golds': devgolds }, jsonfile)
            
step1_dev_set_labels(k=4, step1_iter='0_augment2')

*** step1_iter: 0_augment2 ***
n       4 acc (dev) (since)      dt
1             1.000 (1.000)  2.14 s
2             0.750 (0.500)  2.79 s
4             0.750 (0.750)  4.38 s
8             0.812 (0.875)  7.34 s
16            0.844 (0.875)  12.9 s
32            0.891 (0.938)    24 s
64            0.898 (0.906)  45.6 s
128           0.891 (0.883)  1.47 m
256           0.863 (0.836)   2.9 m
512           0.864 (0.865)  5.78 m
526           0.866 (0.929)  5.95 m


# Step 2 Dev Set Labels

In [ ]:
def step2_dev_set_labels(*, step2_iter, step1_iter, k):
    import json
    from peft import prepare_model_for_kbit_training
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedNewsCat import dev_loader
    from ProgressPrinter import ProgressPrinter
    from transformers import T5ForConditionalGeneration
    import torch
    import warnings
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(8675309)

    dev = dev_loader(batch_size=2)

    t5 = prepare_model_for_kbit_training(T5ForConditionalGeneration.from_pretrained('google/flan-t5-xxl', load_in_8bit=True))
    taskllm_model_id = f'User_keq{k}_t5xxl_step1_iter{step1_iter}'
    t5.load_adapter(taskllm_model_id, 'raw_taskllm')
    t5.load_adapter(taskllm_model_id, 'ema_taskllm')
    rhat_model_id = f'User_keq{k}_t5xxl_step2_iter{step2_iter}'
    t5.load_adapter(rhat_model_id, 'raw_rhat')
    t5.load_adapter(rhat_model_id, 'ema_rhat')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_suffix="taskllm", model_id=taskllm_model_id, choices=dev.choices)
    rewardpredictor = RewardPredictor(t5=t5, adapter_suffix="rhat", model_id=rhat_model_id)

    gumbel = torch.distributions.gumbel.Gumbel(0,1)
    def randomized_similarity(embeddings, nsamples):
        scores = embeddings[0,:] @ embeddings[1:,:].T
        temperature = scores[0].item() - scores[min(scores.shape[0]-1, 4)].item()
        gumbel_shape = torch.Size([nsamples, scores.shape[0]])
        gumbels = temperature * gumbel.sample(gumbel_shape).to(scores.device)
        safek = min(k, scores.shape[0])
        return torch.unique(torch.topk(scores.unsqueeze(0) + gumbels, dim=1, k=safek).indices, sorted=False, dim=0)

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]

    def make_prior(profile):
        from math import log

        c = [1]*len(dev.choices)
        for v in profile:
            c[dev.choices.index(v['category'])] += 1
        n = sum(c)

        return [ log(cnt) - log(n) for cnt in c ]
    
    print(f'*** step1_iter: {step1_iter} step2_iter: {step2_iter} ***')
    nvoters = 1

    devgolds = []
    with ProgressPrinter(f'{k} acc (dev)') as printer, warnings.catch_warnings():
        warnings.filterwarnings("ignore", ".*MatMul8bitLt:.*")
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        for examples, labels in dev:
            with torch.no_grad():
                prior = [ make_prior(ex['profile']) for ex in examples ]
                texts_to_embed = [ [ text[:256]
                                     for text in (' '.join(ex['article'].split()), )
                                   ] +
                                   [ text[:256]
                                     for v in ex['profile']
                                     for text in (' '.join(v['text'].split()), )
                                   ]
                                   for ex in examples
                                 ]
                embeddings = torch.cat(inner_batch(func = dev.embed,
                                                   inner_batch_size = 128,
                                                   inputs = (sum(texts_to_embed, []),)
                                                  ),
                                       dim=0)
                splits = cumsum(map(len, texts_to_embed))
                randos = [ randomized_similarity(embeddings[a:b,:], 64) for a, b in zip(splits, splits[1:]) ]
                prompts = [ [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in indices ])
                              for indices in rando.to('cpu').tolist()
                            ]
                            for ex, rando in zip(examples, randos)
                          ]
                rhats = torch.cat(inner_batch(func = rewardpredictor.predict,
                                              inner_batch_size = 128,
                                              inputs = (sum(prompts, []),)
                                             ),
                                  dim=0)
                splits = cumsum(map(len, prompts))
                votingprompts = [ [ prompt[v] for v in torch.topk(rhats[a:b].view(-1), k=min(nvoters, b-a)).indices.to('cpu').tolist() ]
                                    for a, b, prompt in zip(splits, splits[1:], prompts)
                                ]
                votingpriors = [ [q]*min(nvoters, b-a) for a, b, q in zip(splits, splits[1:], prior) ]
                predicts = torch.cat(inner_batch(func = lambda p, q: taskllm.predict(p, prior=torch.Tensor(q).to(device)),
                                                 inner_batch_size = 128,
                                                 inputs = (sum(votingprompts, []), sum(votingpriors, [])),
                                                ),
                                     dim=0)
                splits = cumsum(map(len, votingprompts))
                guesses = torch.cat([ predicts[a:b,:].logsumexp(dim=0, keepdim=True).argmax(dim=1)
                                      for a, b in zip(splits, splits[1:])
                                    ],
                                    dim=0)

                targets = [ dev.choices.index(label) for label in labels ]
                targets = torch.Tensor(targets).long().to(guesses.device)
                acc = (guesses == targets).float().mean().item()

                for ex, guess in zip(examples, guesses):
                    devgolds.append({ 'id': ex['id'], 'output': dev.choices[guess] })

            printer.addobs(acc)

    with open(f'lamp2u_xxl_step2_dev_golds.json', 'w') as jsonfile:
        json.dump({ 'task': 'LaMP_2', 'golds': devgolds }, jsonfile)
            
step2_dev_set_labels(k=4, step1_iter='0_augment2', step2_iter='0_augment2')

*** step1_iter: 0_augment2 step2_iter: 0_augment2 ***
n       4 acc (dev) (since)      dt
1             1.000 (1.000)  10.3 s
2             0.750 (0.500)  21.1 s
4             0.750 (0.750)  44.7 s
8             0.875 (1.000)  1.32 m
16            0.844 (0.812)  2.28 m
32            0.859 (0.875)  4.43 m
64            0.875 (0.891)  8.55 m
128           0.879 (0.883)  17.2 m
256           0.854 (0.828)  35.1 m
512           0.862 (0.871)  1.18 h
526           0.864 (0.929)  1.21 h
